In [8]:
import requests
import mysql.connector
from mysql.connector import Error
import schedule
import time
 


In [9]:
# Airtable configuration
AIRTABLE_BASE_ID = 'appGnf8ldkBqyxsdX'
AIRTABLE_API_KEY = 'patb92KmkkX08CuWq.3e77882b6467bc2705d5f93a6468e7d74f8276494194126fe99328891d59f46b'
AIRTABLE_URL = f'https://api.airtable.com/v0/{AIRTABLE_BASE_ID}/'

# MySQL configuration
MYSQL_HOST = 'ugandarugbyunion.c5s2s2ki82x4.eu-north-1.rds.amazonaws.com'
MYSQL_DATABASE = 'ugandarugbyunion'
MYSQL_USER = 'admin'
MYSQL_PASSWORD = '2210kpeteR$$'

In [10]:
# Function to fetch records from Airtable
table_name = "teams"
def fetch_airtable_records(table_name):
    headers = {
        'Authorization': f'Bearer {AIRTABLE_API_KEY}',
        'Content-Type': 'application/json'
    }
    response = requests.get(f'{AIRTABLE_URL}{table_name}', headers=headers)
    
    if response.status_code == 200:
        return response.json().get('records', [])
    else:
        print(f"Failed to fetch records from Airtable: {response.status_code}, Response: {response.text}")
        return []
records = fetch_airtable_records(table_name)
print(f"✅ {table_name}: {len(records)} records fetched.")

✅ teams: 92 records fetched.


In [11]:
refs = fetch_airtable_records('referees')  # Specify the table name
print(refs)

[{'id': 'rec3vwG6Jcan8XEMC', 'createdTime': '2025-04-08T07:07:22.000Z', 'fields': {'RefName': 'Ronald Wutimber', 'Status': 'Active\n'}}, {'id': 'rec5Na39g4MocpCKB', 'createdTime': '2025-04-08T07:07:22.000Z', 'fields': {'RefName': 'Emmanuel Sama', 'Status': 'Active\n'}}, {'id': 'rec5Pja7frX2q6x0X', 'createdTime': '2025-04-08T07:07:22.000Z', 'fields': {'RefName': 'Thomas Obajja', 'Status': 'Active\n'}}, {'id': 'rec7Lpd6abka7VfAF', 'createdTime': '2025-04-08T07:07:22.000Z', 'fields': {'RefName': ' Brian Batakwire', 'Status': 'Active\n'}}, {'id': 'rec7jKn3plZ15qaoL', 'createdTime': '2025-04-08T07:07:22.000Z', 'fields': {'RefName': 'Saudah Adiru', 'Status': 'Active\n'}}, {'id': 'recBXzvEvLbsTQd7E', 'createdTime': '2025-04-08T07:07:22.000Z', 'fields': {'RefName': 'Umar Balikanda', 'Status': 'Active\n'}}, {'id': 'recDGN1Fo8YJeGmjl', 'createdTime': '2025-04-08T07:07:22.000Z', 'fields': {'RefName': ' Bashir Bakel', 'Status': 'Active\n'}}, {'id': 'recEGtFE3dwK8yOxu', 'createdTime': '2025-04-08T0

In [12]:
def upsert_record(cursor, table, record):
    fields = record.get('fields', {})

    if table == 'referees':
        sql = """
        INSERT INTO referees (RefName, Status)
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE 
            Status = VALUES(Status);
        """

        values = (
            fields.get('RefName'), 
            fields.get('Status')
        )
        print(f"Executing SQL: {sql} with values: {values}")  # Logging

    else:
        print(f"⚠️ Table '{table}' not handled.")
        return

    try:
        cursor.execute(sql, values)
    except Error as err:
        print(f"Error executing SQL for table {table}: {err}")

In [13]:
def sync_airtable_to_mysql():
    try:
        connection = mysql.connector.connect(
            host=MYSQL_HOST,
            database=MYSQL_DATABASE,
            user=MYSQL_USER,
            password=MYSQL_PASSWORD
        )
        
        if connection.is_connected():
            cursor = connection.cursor()

            # Fetch current records from MySQL for comparison
            cursor.execute("SELECT RefName FROM referees;")
            existing_records = {row[0] for row in cursor.fetchall()}

            # Fetch records from Airtable
            records = fetch_airtable_records('referees')  # Specify the table name
            
            # Prepare to track records that were found in Airtable
            fetched_ref_names = set()

            for record in records:
                ref_name = record['fields'].get('RefName')
                status = record['fields'].get('Status')
                
                # Upsert record into MySQL
                upsert_record(cursor, 'referees', record)

                # Track fetched referee names
                fetched_ref_names.add(ref_name)

            # Deletions: Identify records in MySQL not found in Airtable
            for ref_name in existing_records:
                if ref_name not in fetched_ref_names:
                    # Delete from MySQL if it's no longer in Airtable
                    delete_sql = "DELETE FROM referees WHERE RefName = %s;"
                    cursor.execute(delete_sql, (ref_name,))

            connection.commit()  # Commit all changes
            print("Data synchronized successfully, including deletions.")

    except Error as e:
        print(f"Error while connecting to MySQL: {e}")
    finally:
        if (connection.is_connected()):
            cursor.close()
            connection.close()
# Schedule the sync function to run every minute (adjust as needed)
schedule.every(1).minutes.do(sync_airtable_to_mysql)

# Keep the script running to maintain the scheduling
while True:
    schedule.run_pending()
    time.sleep(1)  # Sleep to prevent high CPU usage


Executing SQL: 
        INSERT INTO referees (RefName, Status)
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE 
            Status = VALUES(Status);
         with values: ('Ronald Wutimber', 'Active\n')
Executing SQL: 
        INSERT INTO referees (RefName, Status)
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE 
            Status = VALUES(Status);
         with values: ('Emmanuel Sama', 'Active\n')
Executing SQL: 
        INSERT INTO referees (RefName, Status)
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE 
            Status = VALUES(Status);
         with values: ('Thomas Obajja', 'Active\n')
Executing SQL: 
        INSERT INTO referees (RefName, Status)
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE 
            Status = VALUES(Status);
         with values: (' Brian Batakwire', 'Active\n')
Executing SQL: 
        INSERT INTO referees (RefName, Status)
        VALUES (%s, %s)
        ON DUPLICATE KEY UPDATE 
            Status = VALUES(Status);
  

MySQLInterfaceError: Lost connection to MySQL server during query